# Clip GeoJSON with HERE XYZ

This example notebook loads a GeoJSON dataset, shows it on a map and lets you select a bounding box for which XYZ will send you the clipped GeoJSON.

In [ ]:
import os
from functools import partial

import geojson
import requests
import warnings
from ipywidgets import Textarea, VBox, Layout
from ipyleaflet import Map, GeoJSON, \
    DrawControl, LayersControl, FullScreenControl, WidgetControl

from xyzspaces.datasets import get_countries_data
from xyzspaces.tools import subset_geojson
from xyzspaces.utils import feature_to_bbox
import xyzspaces

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have XYZ Token to interact with xyzspaces. 
                Please see README.md in notebooks folder for more info on XYZ_TOKEN
</div>

In [ ]:
# Make a XYZ object
try:
    xyz_token = os.environ["XYZ_TOKEN"]
except KeyError:
    xyz_token = "MY-FANCY-XYZ-TOKEN"
    if xyz_token == "MY-FANCY-XYZ-TOKEN":
        warnings.warn(
            "Please either set your actual token to env variable XYZ_TOKEN or "
            "just assign value of your actual token to variable xyz_token above."
        )
xyz = xyzspaces.XYZ(credentials=xyz_token)

## Load GeoJSON dataset

This loads a example GeoJSON dataset from `xyzspaces.datasets`, essentially this one with
minor clean-ups: https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json.

In [ ]:
gj_countries = get_countries_data()

## Create draw control

Using the rectangle widget from the draw control you can select a bounding box defining a subset of the GeoJSON data to be shown in a text area.

In [ ]:
draw_control = DrawControl(position='topright') # position not working, yet
draw_control.polygon = {}
draw_control.polyline = {}
draw_control.circle = {}
draw_control.circlemarker = {}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.1
    }
}

## Show simple map with example dataset

In [ ]:
height = 500
m = Map(center=[0, 0], zoom=2, layout=Layout(width="100%"))
m.add_control(FullScreenControl(position='topright'))
m.add_control(LayersControl(position='topright'))
m.add_control(draw_control)
m += GeoJSON(data=gj_countries, style={'color': 'blue'}, name="World Countries")

In [ ]:
ui = VBox([m])
ui

## Build/show extended map UI

In [ ]:
def on_draw_callback(event, the_map=None, the_textarea=None):
    """Callback function for adding draw controls to a map.
    """
    # TODO: make sure we react only to rectangles...
    # print(event.type)
    if event.name == "last_draw" and event.type == "change":
        if the_textarea:
            the_textarea.value = "Clipping..."
        rect = event.new
        bbox = feature_to_bbox(rect)
        tiled_bbox = subset_geojson(xyz_token, gj_countries, bbox, clip=True)
        if the_map:
            style = dict(color="red")
            the_map.add_layer(GeoJSON(data=tiled_bbox, style=style, name="Foo"))
        if the_textarea:
            the_textarea.value = geojson.dumps(tiled_bbox)
        # draw_control.clear()

In [ ]:
gj_ta = Textarea("GeoJSON of the selected bounding box...", layout=Layout(width="100%", height="200px"))
draw_control.observe(partial(on_draw_callback, the_map=m, the_textarea=gj_ta))

In [ ]:
ui.children = [m, gj_ta]

## Select a bounding box on the map!

Click on the left rectangle button to select a bounding box on the shown dataset for which to create a clipped GeoJSON string in the right textarea! 